In [1]:
from qbraid import device_wrapper, get_devices

In [2]:
get_devices()

Provider,Name,qBraid ID
AWS,Braket Default Simulator,aws_braket_default_sim
AWS,Density Matrix Simulator,aws_dm_sim
AWS,State Vector Simulator,aws_sv_sim
AWS,Tensor Network Simulator,aws_tn_sim
D-Wave,Advantage_system1.1,aws_dwave_advantage_system1
D-Wave,DW_2000Q_6,aws_dwave_2000Q_6
Google,Bristlecone,google_bristlecone
Google,Cirq Density Matrix Simulator,google_cirq_dm_sim
Google,Cirq Sparse Simulator,google_cirq_sparse_sim
Google,Foxtail,google_foxtail


In [ ]:
import cirq
import numpy as np
from braket.circuits import Circuit as BraketCircuit
from qiskit import QuantumCircuit as QiskitCircuit
import networkx as nx
import dwave_networkx as dnx

In [ ]:
n = 5
m = 10
graph = nx.gnm_random_graph(n, m, seed=42)

In [ ]:
# result = dnx.min_vertex_cover(graph, dwave_sampler, answer_mode="histogram")
# print('Result to MVC problem:', result)
# print('Size of the vertex cover:', len(result))

In [ ]:
def braket_circuit():
    circuit = BraketCircuit()
    circuit.h(0)
    circuit.ry(0, np.pi / 2)
    return circuit

In [ ]:
def cirq_circuit(meas=True):
    q0 = cirq.GridQubit(0, 0)
    def test_circuit():
        yield cirq.H(q0)
        yield cirq.Ry(rads=np.pi / 2)(q0)
        if meas:
            yield cirq.measure(q0, key='q0')
    circuit = cirq.Circuit()
    circuit.append(test_circuit())
    return circuit

In [ ]:
def qiskit_circuit(meas=True):
    circuit = QiskitCircuit(1, 1) if meas else QiskitCircuit(1)
    circuit.h(0)
    circuit.ry(np.pi / 2, 0)
    if meas:
        circuit.measure(0, 0)
    return circuit